## Importing required libraries

In [14]:
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pickle
import numpy.ma as ma
import lightgbm as lgb
#from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
#from sklearn.utils import shuffle
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import time
from sklearn.metrics import multilabel_confusion_matrix

In [9]:
merged=pd.read_csv("merged_modified2.csv")
merged=merged.drop(["Unnamed: 0"],axis=1)
merged

,patient_id,1482681,1520176,1520177,1520178,1520179,1520180,1520181,1520182,1520183,...,1_y,2,3_y,4_y,5,6_y,7,8_y,9,10_y
0,16359994,0.883785,1.550000,78.500000,117.0,129.0,NaN,NaN,NaN,16.100000,...,0,0,0,0,0,0,0,0,9,0
1,16360107,2.967860,2.200000,92.000000,38.4,NaN,2454.0,NaN,7.266667,15.500000,...,0,2,0,0,0,0,0,8,0,10
2,16360581,NaN,NaN,96.000000,NaN,NaN,NaN,NaN,NaN,11.500000,...,0,0,0,0,0,0,7,0,0,0
3,16360590,2.304110,NaN,84.500000,NaN,NaN,NaN,NaN,NaN,11.000000,...,0,0,3,0,0,0,0,0,0,0
4,16360714,NaN,NaN,90.000000,NaN,NaN,NaN,NaN,NaN,11.857143,...,0,0,0,0,0,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19998350,NaN,2.100000,81.666667,NaN,32.0,NaN,NaN,NaN,13.333333,...,0,2,0,0,0,0,0,0,0,0
9996,19998497,NaN,1.500000,91.100000,24.0,199.5,NaN,NaN,NaN,15.600000,...,0,2,3,0,0,6,0,0,0,0
9997,19999379,NaN,NaN,92.000000,NaN,NaN,NaN,NaN,NaN,13.000000,...,0,0,3,0,0,6,0,0,0,0
9998,19999625,0.960741,0.100000,85.666667,65.0,NaN,NaN,NaN,NaN,11.833333,...,0,0,0,0,0,0,0,0,0,10


In [ ]:
%%time
list_patients=merged["patient_id"].unique()

list_original=[]
list_prediction=[]
list_prediction_with_zero=[]
list_original_with_zero=[]
list_how_many_patients=[]

prediction_arr=[]
y_test_arr=[]

for i in range(len(list_patients)):
    
    
    # get the start time
    st = time.time()
    
    
#     if i>299: #doing for only first 300 patients for now
#         break
    #print("Patient:",i+1)
    patient_id=list_patients[i]
    test_set=merged.query("patient_id == @patient_id")
    test_set = test_set.dropna(how='all', axis=1)
    #display(test_set)
    list_columns=test_set.columns.tolist()

    n_columns=len(test_set.columns)
    X_test=test_set.iloc[:,1:n_columns-10]
    X_test = X_test.dropna(how='all', axis=1)
    #display(X_test) #Its ok

    n_col=len(X_test.columns)
    if i>0:
        print("\n")
    print("Number of features found for Patient",i+1, n_col)



    X_test=X_test.values
    a=X_test
    #X_test=np.where(pd.isnull(a), ma.array(a, mask=pd.isnull(a)).mean(axis=0), a)  
    y_test=test_set.iloc[:,n_columns-10:n_columns]
    y_test_actual=y_test
    #print(y_test_actual)
    y_test_actual=y_test_actual.values
    y_test=y_test.replace({2:1,3:1,4:1,5:1,6:1,7:1,8:1,9:1,10:1},regex=True)
    #display(y_test)
    y_test=y_test.values    
    
    
    training_set=merged.query("patient_id != @patient_id")
    #training_set=training_set.iloc[:,1:n_columns-10]
    training_set=training_set[list_columns]

    #training_set=training_set.dropna(thresh=12)
    training_set=training_set.dropna()
    training_set=training_set.reset_index()
    training_set=training_set.drop(['index'], axis=1)
    #display(training_set)
    
#     length_n=len(list_columns)
#     #print(length_n)
#     list_columns_modi=list_columns[1:length_n-10]
#     list_columns_modi
#     len(list_columns_modi)
#     training_set=training_set.dropna(how=list_columns_modi,axis=0)


    n_columns=len(training_set.columns)
    y_train=training_set.iloc[:,n_columns-10:n_columns]
    y_train=y_train.replace({2:1,3:1,4:1,5:1,6:1,7:1,8:1,9:1,10:1},regex=True)
    #display(y_train)
    y_train=y_train.values
    #y_train.shape

#     #y_test.shape


    X_train=training_set.iloc[:,1:n_columns-10]
    #display(X_train)
    print("Number of patients in Patient Cohort:", X_train.shape[0])
    
    if X_train.shape[0]>=1: #10*n_col:
        list_how_many_patients.append(i)
        X_train=X_train.values
        a=X_train
        #X_train=np.where(pd.isnull(a), ma.array(a, mask=pd.isnull(a)).mean(axis=0), a)  
        #X_train.shape


        #X_test.shape

#         oversample = SMOTE()
#         X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)
        
        # Feature Scaling 
#         sc_X = RobustScaler()
#         X_train = sc_X.fit_transform(X_train)
#         X_test = sc_X.transform(X_test)


        n_samples, n_features = X_train.shape # 10000,605
        n_outputs = y_train.shape[1] # 10
        n_classes = 11
        
        TOTAL_TREES=int(n_col/3)
        if TOTAL_TREES>64:
            TOTAL_TREES=TOTAL_TREES
        else:
            TOTAL_TREES=64
        
#         grid = { 'n_estimators': [200,300,400,500],
#                  'max_features': ['sqrt', 'log2'],
#                  'max_depth' : [4,5,6,7,8],
#                  'criterion' :['gini', 'entropy'],
#                  'random_state' : [18]}
#         grid={'n_jobs': [-1, -2, -3, -4, 0]}
#         print("grid given")

#         multi_cv = GridSearchCV(estimator=MultiOutputClassifier(RandomForestClassifier()), param_grid=grid)
#         multi_cv.fit(X_train, y_train)
        
#         print(multi_cv.best_params_)
        
#         rf_cv = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid)
#         multi_target_forest = MultiOutputClassifier((rf_cv), n_jobs=-1)

#         multi_target_forest.fit(X_train, y_train)


#         forest = RandomForestClassifier(n_estimators=TOTAL_TREES, random_state=1234, n_jobs=multi_cv.best_params_['n_jobs'])
#         multi_target_forest = MultiOutputClassifier(forest, n_jobs=multi_cv.best_params_['n_jobs'])
#         prediction=multi_target_forest.fit(X_train, y_train).predict(X_test)


        forest = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))

        multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)
        prediction=multi_target_forest.fit(X_train, y_train).predict(X_test)
#         prediction_probability=trained_model.predict_proba(X_test)
        
#         print("For patient", i+1, ":")
#         for k in range(len(prediction_probability)):
#             #for l in range(len(prediction_probablity[k])):
#             print("The probability of having Diagnosis",k+1,"is",prediction_probability[k][0][1]*100,"%")
        
        #print(prediction[0])
        y_test_arr.append(y_test[0].tolist())
        prediction_arr.append(prediction[0].tolist())
        for i in range(y_test.shape[1]):
            list_original_with_zero.append(y_test_actual[0][i])
            if prediction[0][i]==1:
                list_prediction_with_zero.append(y_test_actual[0][i])
            else:
                list_prediction_with_zero.append(0)
            
            if y_test[0][i]!=0:
                list_original.append(y_test_actual[0][i])
                if prediction[0][i]==1:
                    list_prediction.append(y_test_actual[0][i])
                else:
                    list_prediction.append(0)
                    
    else:
          print("Number of patients found in the cohort for same concepts for this patient is less than 1, ignoring training model")
            
    # get the end time
    et = time.time()
    
    # get the execution time
    elapsed_time = et - st
    print('Execution time:', elapsed_time, 'seconds')

In [22]:
y_true = np.array(y_test_arr)
y_pred = np.array(prediction_arr)

#print(classification_report(y_true, y_pred, target_names=labels))
conf_matrix=multilabel_confusion_matrix(y_true, y_pred)
conf_matrix

array([[[4758,  598],
        [1056,  276]],

       [[4952,  577],
        [ 942,  217]],

       [[5168,  236],
        [ 925,  359]],

       [[5084,  933],
        [ 342,  329]],

       [[6337,  166],
        [ 111,   74]],

       [[5515,  150],
        [ 742,  281]],

       [[5571,  428],
        [ 577,  112]],

       [[5989,  158],
        [ 466,   75]],

       [[5675,  219],
        [ 751,   43]],

       [[5682,  427],
        [ 498,   81]]])

In [23]:
Diagnosis_list=[]
Precision_list=[]
Recall_list=[]
F1_list=[]
Support_list=[]
TTP=[]
TTN=[]
TFP=[]
TFN=[]

for i in range(len(conf_matrix)):
    #print(i)
    Diagnosis_list.append(i)
    conf=conf_matrix[i]
    TP  = conf[0][0]
    TN   = conf[0][1]
    FP  = conf[1][0]
    FN  = conf[1][1]
    TTP.append(TP)
    TTN.append(TN)
    TFP.append(FP)
    TFN.append(FN)
    #print(TP, TN, FP, FN)
    Precision = TP / (TP + FP)
    Recall    = TP / (TP + FN)
    F1=2*Precision*Recall/(Precision+Recall)
    
    
#     list_name='list_'+str(i)
#     #print(list_name)
#     myVars = vars()
#     myVars[list_name] = []
#     for j in range(len(y_true)):
#         if y_true[j][i]==1:
#             myVars[list_name].append(1)
#     Support=len(myVars[list_name])
#     #print(Support)

    Precision_list.append(Precision)
    Recall_list.append(Recall)
    F1_list.append(F1)
    #Support_list.append(Support)
    Support_list.append(len(list_how_many_patients))


Precision_series=pd.Series(Precision_list)
Recall_series=pd.Series(Recall_list)
F1_series=pd.Series(F1_list)
Support_series=pd.Series(Support_list)


frame = {'Diagnosis': Diagnosis_list,'Precision': Precision_series, 'Recall': Recall_series, 'F1 Score': F1_series, 'Support': Support_series }
result = pd.DataFrame(frame)
#Printing elements of Dataframe
result

,Diagnosis,Precision,Recall,F1 Score,Support
0,0,0.818369,0.945173,0.877212,6688
1,1,0.840176,0.958019,0.895236,6688
2,2,0.848186,0.935046,0.889501,6688
3,3,0.936970,0.939220,0.938094,6688
4,4,0.982785,0.988457,0.985613,6688
5,5,0.881413,0.951518,0.915125,6688
6,6,0.906148,0.980292,0.941763,6688
7,7,0.927808,0.987632,0.956786,6688
8,8,0.883131,0.992480,0.934618,6688
9,9,0.919417,0.985945,0.951520,6688


In [24]:
## second result
Precision_list=[]
Recall_list=[]
F1_list=[]
Support_list=[]

list_section=['micro avg','macro avg','weighted avg']
for i in range(len(list_section)):
    #print(i)
    if list_section[i]=='micro avg':
        ##micro average
        TP=sum(TTP)
        TN=sum(TTN)
        FP=sum(TFP)
        FN=sum(TFN)

        Precision = TP / (TP + FP)
        Recall    = TP / (TP + FN)
        F1=2*Precision*Recall/(Precision+Recall)
        Support=int((Support_series.sum())/10)
        
        Precision_list.append(Precision)
        Recall_list.append(Recall)
        F1_list.append(F1)
        Support_list.append(Support)
        
    elif list_section[i]=='macro avg':
        Precision = Precision_series.mean()
        Recall    = Recall_series.mean()
        F1=F1_series.mean()
        Support=int((Support_series.sum())/10)
        
        Precision_list.append(Precision)
        Recall_list.append(Recall)
        F1_list.append(F1)
        Support_list.append(Support)
        
    elif list_section[i]=='weighted avg':
        Precision = (Precision_series*Support_series/Support_series.sum()).sum()
        Recall    = (Recall_series*Support_series/Support_series.sum()).sum()
        F1=(F1_series*Support_series/Support_series.sum()).sum()
        Support=int((Support_series.sum())/10)
        
        Precision_list.append(Precision)
        Recall_list.append(Recall)
        F1_list.append(F1)
        Support_list.append(Support)      
        
        
Precision_series=pd.Series(Precision_list)
Recall_series=pd.Series(Recall_list)
F1_series=pd.Series(F1_list)
Support_series=pd.Series(Support_list)


frame = {'Section': pd.Series(list_section),'Precision': Precision_series, 'Recall': Recall_series, 'F1 Score': F1_series, 'Support': Support_series }
result = pd.DataFrame(frame)
#Printing elements of Dataframe
result





,Section,Precision,Recall,F1 Score,Support
0,micro avg,0.895160,0.967355,0.929858,6688
1,macro avg,0.894441,0.966378,0.928547,6688
2,weighted avg,0.894441,0.966378,0.928547,6688
